#### Import the required packages

In [8]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import os
import csv
import subprocess

%matplotlib inline

In [2]:
from ripser import ripser
from persim.landscapes import (
PersLandscapeApprox,
average_approx,
snap_pl,
plot_landscape,
plot_landscape_simple
)

#### The persistence data are computed and stored in data folder

In [5]:
directory_in_str = '../data/annotated_PH_2018_500/'
directory = os.fsencode(directory_in_str)

folders_region1 = []

for folder in os.listdir(directory):
    foldername = os.fsdecode(folder)
    if foldername.startswith('PH_640_'):
        folders_region1.append(foldername)
        foldername = foldername

folders_region2 = []

for folder in os.listdir(directory):
    foldername = os.fsdecode(folder)
    if foldername.startswith('PH_3_'):
        folders_region2.append(foldername)
        foldername = foldername

#### Get persistence data stored in files within the folder whose location is passed through the keyword argument path

In [6]:
number_of_edges_threshold = 0

"""
Returns persistence data
Inputs -- path: local_path
       -- starts_with: the string that the filenames start with if needed
       -- ends_with: the string that the filenames end with if needed
       -- threshold: number of edges threshold of the underlying vascular networks
"""

def get_data(path=None,starts_with=None,ends_with=None,threshold=None):
    files = [os.fsdecode(file) for file in os.listdir(path)]
    persistence_data={}
    for filename in files:
        if starts_with and ends_with:
            if filename.startswith(starts_with) and filename.endswith(ends_with):
                data = []
                with open(path+filename) as f:
                    reader = csv.reader(f)
                    for row in reader:
                        data.append([float(x) for x in row])
                if threshold == 0:
                    persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
                else:
                    if data[0][1] >= threshold:
                        persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
        elif starts_with:
            if filename.startswith(starts_with):
                data = []
                with open(path+filename) as f:
                    reader = csv.reader(f)
                    for row in reader:
                        data.append([float(x) for x in row])
                if threshold == 0:
                    persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
                else:
                    if data[0][1] >= threshold:
                        persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
        elif ends_with:
            if filename.endswith(ends_with):
                data = []
                with open(path+filename) as f:
                    reader = csv.reader(f)
                    for row in reader:
                        data.append([float(x) for x in row])
                if threshold == 0:
                    persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
                else:
                    if data[0][1] >= threshold:
                        persistence_data[filename]=[item for item in data[1:] if item[1]!=np.inf]
                
    return persistence_data

#### For comparitive analysis between vascular networks in bulk of mouse brain tissue -- region-1 vs. region-2

In [9]:
r1_PHs={}
for folder in folders_region1:
    r1_PHs[folder] = get_data('../data/annotated_PH_2018_500/'+folder+'/',starts_with=('ph_'),threshold=number_of_edges_threshold)

r2_PHs={}
for folder in folders_region2:
    r2_PHs[folder] = get_data('../data/annotated_PH_2018_500/'+folder+'/',starts_with=('ph_'),threshold=number_of_edges_threshold)

#### Use the following helper function to get persistence data, which is stored within a dictionary of dictinoaries.

In [10]:
"""
Input -- persistence_data: a dictionary of folders consisting of dictionaries 
         of files whose values are the lists of persistence (b,d) pairs dataset
"""

def get_pl(persistence_data):
    pl_data = []
    for key,value in persistence_data.items():
        if value:
            for subkey,subvalue in value.items():
                pl_data.append(subvalue)
                if subvalue == []:
                    print(subkey)
    return pl_data

#### Loading the persistence data, downsampling (optional) and creating the labels for classification

In [11]:
r1_ph_dataset = get_pl(r1_PHs)
r2_ph_dataset = get_pl(r2_PHs)

## Downsampling
# indices = np.random.randint(0,len(r2_ph_dataset),len(r1_ph_dataset))
# r2_ph_dataset = [r2_ph_dataset[k] for k in indices]

y=[]
for k in range(len(r1_ph_dataset)+len(r2_ph_dataset)):
    if k < len(r1_ph_dataset):
        y.append(0)
    elif k >= len(r1_ph_dataset):
        y.append(1)
        
dataset = [np.array(item) for item in r1_ph_dataset+r2_ph_dataset]

## Check that the number of samples is consistent depending on whether or not the downsampling is applied
print((sum(y)/len(y),len(y)))

(0.6739130434782609, 322)


#### Train-test split

In [12]:
from sklearn.model_selection import train_test_split
## for balanced dataset
from sklearn.metrics import accuracy_score
## for imbalanced dataset
from sklearn.metrics import f1_score

## while doing cross-validation, this is optional
ph_train, ph_test, y_train, y_test = train_test_split(dataset, y,
                                                      test_size = 0.1,
                                                      random_state=123,
                                                      shuffle=True,
                                                      stratify=y)

#### Compute approimate persistence landscapes

In [39]:
approx_pl_train_dataset=[]
for entry in ph_train:
    approx_pl_train_dataset.append(PersLandscapeApprox(dgms=[np.array([[item[0],item[1]] for item in entry])],hom_deg=0))

In [41]:
approx_pl_test_dataset=[]
for entry in ph_test:
    approx_pl_test_dataset.append(PersLandscapeApprox(dgms=[np.array([[item[0],item[1]] for item in entry])],hom_deg=0))

#### Compute the average of the approximate persistence landscapes for each class

In [64]:
avg_pl_train_region2 = average_approx([approx_pl_train_dataset[k] for k in np.squeeze(np.argwhere(y_train))])
avg_pl_train_region1 = average_approx([approx_pl_train_dataset[j] for j in \
                                       [k for k in range(len(y_train)) if k not in np.squeeze(np.argwhere(y_train))]])

#### Compute the distance between the average approximate persistence landscape for the vascular networks in two regions

In [67]:
[avg_pl_train_region1_snapped, avg_pl_train_region2_snapped] = snap_pl([avg_pl_train_region1, avg_pl_train_region2])
print(f'Sup-norm of the difference between BS and CH: {(avg_pl_train_region1_snapped-avg_pl_train_region2_snapped).sup_norm()}')
print(f'l1 norm of the difference between BS and CH: {(avg_pl_train_region1_snapped-avg_pl_train_region2_snapped).p_norm(1)}')
print(f'l2 norm of the difference between BS and CH: {(avg_pl_train_region1_snapped-avg_pl_train_region2_snapped).p_norm(2)}')

Sup-norm of the difference between BS and CH: 0.00975169228356082
l1 norm of the difference between BS and CH: 0.06497366310909727
l2 norm of the difference between BS and CH: 0.017962061813394953


#### Use the distances from the averages of approximate persistence landscapes to classify samples from test dataset

In [75]:
sup_distance = np.zeros((len(approx_pl_test_dataset),2))
l1_distance = np.zeros((len(approx_pl_test_dataset),2))
l2_distance = np.zeros((len(approx_pl_test_dataset),2))
for k,sample in enumerate(approx_pl_test_dataset):
    [avg_pl_train_snapped_region1, test]=snap_pl([avg_pl_train_region1, sample])
    t1=avg_pl_train_snapped_region1 - test
    [avg_pl_train_snapped_region2, test]=snap_pl([avg_pl_train_region2, sample])
    t2=avg_pl_train_snapped_region2 - test
    sup_distance[k,0] = t1.sup_norm()
    sup_distance[k,1] = t2.sup_norm()
    l1_distance[k,0] = t1.p_norm(1)
    l1_distance[k,1] = t2.p_norm(1)
    l2_distance[k,0] = t1.p_norm(2)
    l2_distance[k,1] = t2.p_norm(2)

#### The f1-scores for three different choice of norms

In [86]:
print((f1_score(y_test,np.argmin(l1_distance,axis=1)),\
       f1_score(y_test,np.argmin(l2_distance,axis=1)),\
       f1_score(y_test,np.argmin(sup_distance,axis=1))))

(0.5454545454545455, 0.6486486486486486, 0.7368421052631579)
